<a href="https://colab.research.google.com/github/Agniva2006/Implmenting-Blip-From-Scratch-on-mnist-dataset/blob/main/blip_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision tqdm pillow pycocotools matplotlib git+https://github.com/salaniz/pycocoevalcap


  Cloning https://github.com/salaniz/pycocoevalcap to /tmp/pip-req-build-8xwg2dy_
  Running command git clone --filter=blob:none --quiet https://github.com/salaniz/pycocoevalcap /tmp/pip-req-build-8xwg2dy_
  Resolved https://github.com/salaniz/pycocoevalcap to commit a24f74c408c918f1f4ec34e9514bc8a76ce41ffd
  Preparing metadata (setup.py) ... done
  Created wheel for pycocoevalcap: filename=pycocoevalcap-1.2-py3-none-any.whl size=104312245 sha256=313b43728ebf7b18a0ae5e4e8a5411a73b4d01b2e364948801f9a9e8c51337d6
  Stored in directory: /tmp/pip-ephem-wheel-cache-bc_8ml01/wheels/03/ce/0b/3d3fdeecb09b4f4ebcfb3ff28d27a9f5b3c1a7b73897ad122d
Successfully built pycocoevalcap


In [ ]:
import os, json, random, math
import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt


In [ ]:
import os
from pathlib import Path

# Base directory for COCO dataset
coco_dir = Path("/content/coco")
img_dir = coco_dir / "train2017"
ann_dir = coco_dir / "annotations"

# Create folders
os.makedirs(img_dir, exist_ok=True)
os.makedirs(ann_dir, exist_ok=True)

# Download COCO 2017 captions annotation
if not (ann_dir / "captions_train2017.json").exists():
    !wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip -P /content/
    !unzip -q /content/annotations_trainval2017.zip -d /content/coco/
    print(" COCO captions annotations downloaded and extracted!")

# Download COCO train images
if not any(img_dir.iterdir()):
    print(" Downloading COCO 2017 train images (~18GB)... This can take a while.")
    !wget http://images.cocodataset.org/zips/train2017.zip -P /content/
    !unzip -q /content/train2017.zip -d /content/coco/
    print(" COCO train images extracted!")

print("COCO dataset setup complete!")


--2025-10-20 15:05:05--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 16.15.220.78
Connecting to images.cocodataset.org (images.cocodataset.org)|16.15.220.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘/content/annotations_trainval2017.zip’

annotations_trainva 100%[===================>] 241.19M  59.6MB/s    in 4.5s    

2025-10-20 15:05:10 (53.8 MB/s) - ‘/content/annotations_trainval2017.zip’ saved [252907541/252907541]

 COCO captions annotations downloaded and extracted!
--2025-10-20 15:05:22--  http://images.cocodataset.org/zips/train2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.53.89, 54.231.160.97, 3.5.27.72, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.53.89|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19336861798 (18G) [application/

In [ ]:
from PIL import Image
import torch
import os
from transformers import AutoTokenizer
import json

class CocoCaptionsDataset(torch.utils.data.Dataset):
    def __init__(self, img_dir, ann_path, tokenizer_name="bert-base-uncased", transform=None, max_images=None):
        self.img_dir = img_dir
        self.transform = transform
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

        # Load annotations
        with open(ann_path, 'r') as f:
            data = json.load(f)

        # Map image_id -> filename
        self.image_map = {img['id']: img['file_name'] for img in data['images']}

        # Keep only first max_images images (if specified)
        if max_images:
            first_ids = sorted(self.image_map.keys())[:max_images]
            self.annotations = [ann for ann in data['annotations'] if ann['image_id'] in first_ids]
            self.image_map = {k:v for k,v in self.image_map.items() if k in first_ids}
        else:
            self.annotations = data['annotations']

        # Filter annotations whose images actually exist
        self.annotations = [ann for ann in self.annotations
                            if os.path.exists(os.path.join(self.img_dir, self.image_map[ann['image_id']]))]

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        ann = self.annotations[idx]
        img_id = ann['image_id']
        img_name = self.image_map[img_id]
        img_path = os.path.join(self.img_dir, img_name)

        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        caption = ann['caption']
        tokens = self.tokenizer(
            caption,
            padding='max_length',
            truncation=True,
            max_length=40,
            return_tensors='pt'
        )

        return image, tokens["input_ids"].squeeze(0), tokens["attention_mask"].squeeze(0)


In [ ]:
# Paths
IMG_DIR = "/content/coco/train2017"
ANN_PATH = "/content/coco/annotations/captions_train2017.json"

# Dataset init
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

dataset = CocoCaptionsDataset(
    img_dir=IMG_DIR,
    ann_path=ANN_PATH,
    transform=transform,
    max_images=10000
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
class PatchEmbedding(nn.Module):
    def __init__(self, img_size=224, patch_size=16, in_ch=3, embed_dim=512):
        super().__init__()
        self.n_patches = (img_size // patch_size) ** 2
        self.proj = nn.Conv2d(in_ch, embed_dim, patch_size, patch_size)

    def forward(self, x):
        x = self.proj(x).flatten(2).transpose(1,2)
        return x

class MultiHeadSelfAttention(nn.Module):
    def __init__(self, dim, heads):
        super().__init__()
        self.heads=heads; self.head_dim=dim//heads
        self.qkv=nn.Linear(dim,dim*3); self.proj=nn.Linear(dim,dim)
    def forward(self,x):
        B,N,D=x.shape
        qkv=self.qkv(x).reshape(B,N,3,self.heads,self.head_dim).permute(2,0,3,1,4)
        q,k,v=qkv
        attn=(q@k.transpose(-2,-1))/math.sqrt(self.head_dim)
        attn=attn.softmax(dim=-1)
        out=(attn@v).transpose(1,2).reshape(B,N,D)
        return self.proj(out)

class TransformerBlock(nn.Module):
    def __init__(self, dim, heads, mlp_ratio=4.0):
        super().__init__()
        self.norm1=nn.LayerNorm(dim)
        self.attn=MultiHeadSelfAttention(dim,heads)
        self.norm2=nn.LayerNorm(dim)
        self.mlp=nn.Sequential(nn.Linear(dim,int(dim*mlp_ratio)),nn.GELU(),
                               nn.Linear(int(dim*mlp_ratio),dim))
    def forward(self,x):
        x=x+self.attn(self.norm1(x))
        x=x+self.mlp(self.norm2(x))
        return x

class VisionTransformer(nn.Module):
    def __init__(self,img_size=224,patch=16,embed_dim=512,depth=6,heads=8):
        super().__init__()
        self.patch=PatchEmbedding(img_size,patch,3,embed_dim)
        self.cls=nn.Parameter(torch.zeros(1,1,embed_dim))
        self.pos=nn.Parameter(torch.randn(1,1+self.patch.n_patches,embed_dim))
        self.blocks=nn.Sequential(*[TransformerBlock(embed_dim,heads) for _ in range(depth)])
        self.norm=nn.LayerNorm(embed_dim)
    def forward(self,x):
        B=x.size(0)
        x=self.patch(x)
        cls=self.cls.expand(B,-1,-1)
        x=torch.cat([cls,x],1)
        x=x+self.pos[:,:x.size(1)]
        x=self.blocks(x)
        return self.norm(x)[:,0]


In [ ]:
class SelfAttention(nn.Module):
    def __init__(self, dim, heads):
        super().__init__()
        self.heads=heads; self.head_dim=dim//heads
        self.qkv=nn.Linear(dim,dim*3); self.proj=nn.Linear(dim,dim)
    def forward(self,x):
        B,N,D=x.shape
        qkv=self.qkv(x).reshape(B,N,3,self.heads,self.head_dim).permute(2,0,3,1,4)
        q,k,v=qkv
        attn=(q@k.transpose(-2,-1))/math.sqrt(self.head_dim)
        attn=attn.softmax(-1)
        out=(attn@v).transpose(1,2).reshape(B,N,D)
        return self.proj(out)

class CrossAttention(nn.Module):
    def __init__(self, dim, heads):
        super().__init__()
        self.heads=heads; self.head_dim=dim//heads
        self.q=nn.Linear(dim,dim); self.kv=nn.Linear(dim,dim*2); self.proj=nn.Linear(dim,dim)
    def forward(self,q_in,kv_in):
        B,Nq,D=q_in.shape; Nk=kv_in.size(1)
        q=self.q(q_in).reshape(B,Nq,self.heads,self.head_dim).permute(0,2,1,3)
        kv=self.kv(kv_in).reshape(B,Nk,2,self.heads,self.head_dim).permute(2,0,3,1,4)
        k,v=kv
        attn=(q@k.transpose(-2,-1))/math.sqrt(self.head_dim)
        attn=attn.softmax(-1)
        out=(attn@v).transpose(1,2).reshape(B,Nq,D)
        return self.proj(out)

class EncoderLayer(nn.Module):
    def __init__(self,dim,heads):
        super().__init__()
        self.norm1=nn.LayerNorm(dim); self.attn=SelfAttention(dim,heads)
        self.norm2=nn.LayerNorm(dim); self.ff=nn.Sequential(nn.Linear(dim,dim*4),nn.GELU(),nn.Linear(dim*4,dim))
    def forward(self,x):
        x=x+self.attn(self.norm1(x)); x=x+self.ff(self.norm2(x)); return x

class DecoderLayer(nn.Module):
    def __init__(self,dim,heads):
        super().__init__()
        self.self_attn=SelfAttention(dim,heads)
        self.cross=CrossAttention(dim,heads)
        self.norm1=nn.LayerNorm(dim); self.norm2=nn.LayerNorm(dim); self.norm3=nn.LayerNorm(dim)
        self.ff=nn.Sequential(nn.Linear(dim,dim*4),nn.GELU(),nn.Linear(dim*4,dim))
    def forward(self,x,mem):
        x=x+self.self_attn(self.norm1(x))
        x=x+self.cross(self.norm2(x),mem)
        x=x+self.ff(self.norm3(x))
        return x

class TextTransformer(nn.Module):
    def __init__(self,vocab,dim=512,depth=4,heads=8,max_len=40):
        super().__init__()
        self.embed=nn.Embedding(vocab,dim)
        self.pos=nn.Parameter(torch.randn(1,max_len,dim))
        self.enc=nn.Sequential(*[EncoderLayer(dim,heads) for _ in range(depth)])
        # Modify the decoder to be a ModuleList instead of Sequential
        self.dec=nn.ModuleList([DecoderLayer(dim,heads) for _ in range(depth)])
        self.norm=nn.LayerNorm(dim)
        self.fc=nn.Linear(dim,vocab)
    def forward(self,src,tgt):
        src=self.embed(src)+self.pos[:,:src.size(1)]
        mem=self.enc(src)
        tgt=self.embed(tgt)+self.pos[:,:tgt.size(1)]
        # Manually iterate through decoder layers
        out = tgt
        for layer in self.dec:
            out = layer(out, mem)
        return self.fc(self.norm(out)), mem

In [ ]:
class BLIP(nn.Module):
    def __init__(self,vocab,dim=512):
        super().__init__()
        self.vit=VisionTransformer(embed_dim=dim)
        self.txt=TextTransformer(vocab,dim)
        self.img_proj=nn.Linear(dim,dim)
        self.txt_proj=nn.Linear(dim,dim)
        self.temp=nn.Parameter(torch.tensor(0.07))
    def forward(self,img,src,tgt):
        img_feat=self.img_proj(self.vit(img))
        txt_out, mem=self.txt(src,tgt)
        txt_feat=self.txt_proj(mem[:,0])
        img_feat=F.normalize(img_feat,dim=-1)
        txt_feat=F.normalize(txt_feat,dim=-1)
        logits=(img_feat@txt_feat.t())/self.temp
        return logits, txt_out

def blip_loss(logits,txt_out,tgt):
    labels=torch.arange(len(logits)).to(logits.device)
    ce=nn.CrossEntropyLoss(ignore_index=0)
    itc=(ce(logits,labels)+ce(logits.t(),labels))/2
    lm=ce(txt_out.reshape(-1,txt_out.size(-1)),tgt.reshape(-1))
    return itc+lm


In [ ]:

loader=DataLoader(dataset,batch_size=8,shuffle=True,num_workers=2)

vocab_size=30522  # from bert-base-uncased
device="cuda" if torch.cuda.is_available() else "cpu"

model=BLIP(vocab_size).to(device)
opt=torch.optim.AdamW(model.parameters(),lr=5e-5)




In [ ]:
for epoch in range(3):
    model.train()
    total_loss = 0
    for batch_idx, (img, src, mask) in enumerate(tqdm(loader)):
        img, src, mask = img.to(device), src.to(device), mask.to(device)
        logits, txt_out = model(img, src, src)
        loss = blip_loss(logits, txt_out, src)

        opt.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        opt.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}: avg loss {total_loss/len(loader):.4f}")

  4%|▍         | 243/6254 [00:32<13:16,  7.55it/s]


KeyboardInterrupt: 

In [ ]:
!pip install gradio
import gradio as gr
from PIL import Image




In [ ]:
from transformers import AutoTokenizer

# Use the same tokenizer as training
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

device = "cuda" if torch.cuda.is_available() else "cpu"

# Ensure your trained model is loaded
model = model.to(device)
model.eval()

BLIP(
  (vit): VisionTransformer(
    (patch): PatchEmbedding(
      (proj): Conv2d(3, 512, kernel_size=(16, 16), stride=(16, 16))
    )
    (blocks): Sequential(
      (0): TransformerBlock(
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (attn): MultiHeadSelfAttention(
          (qkv): Linear(in_features=512, out_features=1536, bias=True)
          (proj): Linear(in_features=512, out_features=512, bias=True)
        )
        (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (0): Linear(in_features=512, out_features=2048, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=2048, out_features=512, bias=True)
        )
      )
      (1): TransformerBlock(
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (attn): MultiHeadSelfAttention(
          (qkv): Linear(in_features=512, out_features=1536, bias=True)
          (proj): Linear(in_features=512

In [ ]:
import torch
from torchvision import transforms
from PIL import Image
import gradio as gr
import numpy as np
import torch


In [ ]:
def generate_caption(model, image_tensor, tokenizer, max_len=40, device="cuda"):
    model.eval()
    image_tensor = image_tensor.to(device)

    with torch.no_grad():
        # Start token
        start_token = tokenizer.cls_token_id
        input_ids = torch.tensor([[start_token]], dtype=torch.long, device=device)

        # Encode image
        image_feat = model.vit(image_tensor)               # [batch, dim]
        image_feat = model.img_proj(image_feat).unsqueeze(1)  # [batch, seq_len=1, dim]

        for _ in range(max_len):
            logits, _ = model.txt(image_feat, input_ids)  # src=image_feat, tgt=input_ids

            # Get next token
            next_token = logits[:, -1, :].argmax(-1)       # [batch]
            next_token = next_token.long().to(device)      # ensure LongTensor
            input_ids = torch.cat([input_ids, next_token.unsqueeze(1)], dim=1)

            if next_token.item() == tokenizer.sep_token_id:
                break

        caption = tokenizer.decode(input_ids[0], skip_special_tokens=True)
        return caption


In [ ]:
from PIL import Image
import numpy as np

def infer_caption(img):
    if isinstance(img, np.ndarray):
        img = Image.fromarray(img.astype('uint8')).convert('RGB')
    img_tensor = transform(img).unsqueeze(0).to(device)
    caption = generate_caption(model, img_tensor, tokenizer)
    return caption


In [ ]:
gr.close_all()  # Clears old Gradio sessions to prevent event loop errors

demo = gr.Interface(
    fn=infer_caption,
    inputs="image",
    outputs="text",
    title="BLIP-from-Scratch Caption Demo",
    description="Upload an image to generate a caption using your BLIP model."
)

demo.launch(share=True, debug=True)

Closing server running on port: 7860
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a6f858cddc4bad0037.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1133, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://a6f858cddc4bad0037.gradio.live
